<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# 警告：此文件不属于训练营教学步骤。它是一个中高级示例，展示你将学习的内容。如果你是为了学习 Chisel 而参加训练营，不要从这里开始。请从[Scala简介](1_intro_to_scala.ipynb)开始。

# Chisel 演示
**Next: [Introduction to Scala](1_intro_to_scala.ipynb)**

欢迎！也许你是一名听说过“Chisel”这个名字的感兴趣的学生，或者你是一名经验丰富的硬件设计老手，被经理指派探索 Chisel 作为新的 HDL 替代品。无论哪种情况，如果你是 Chisel 新手，你都希望尽快弄清楚所有的事情。别再找了 - 让我们看看 Chisel 有什么可以提供的！

## 设置
在开始之前，我们需要下载并导入演示所需的依赖项。

**请通过按键盘上的 SHIFT+ENTER 或菜单中的运行按钮运行以下两个单元格。**

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

## Hardware Generators: Type-Safe Meta-Programming for RTL

所有硬件描述语言都支持编写单个 RTL 设计实例 - Chisel 也不例外。
事实上，大多数 Verilog/VHDL 数字逻辑设计都可以直接转录到 Chisel 中！
虽然 Chisel 提供了其他很棒的功能，但我们希望强调的是，切换到 Chisel 的用户将保留与任何其他硬件语言相同的设计控制权。

以下是一个以 FIR 滤波器风格实现的 3 点移动平均值的示例。

<img src="images/demo_fir_filter.svg" width="512" />

Chisel 提供了与可综合 Verilog 类似的基本原语，并且*可以*这样使用！运行下一个单元格来声明我们的 Chisel 模块。

In [ ]:
// 3-point moving average implemented in the style of a FIR filter
class MovingAverage3(bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val z1 = RegNext(io.in) // Create a register whose input is connected to the argument io.in
  val z2 = RegNext(z1)    // Create a register whose input is connected to the argument z1

  io.out := (io.in * 1.U) + (z1 * 1.U) + (z2 * 1.U) // `1.U` is an unsigned literal with value 1
}

After defining `class MovingAverage3`, let's instantiate it and take a look at its structure:

In [ ]:
// same 3-point moving average filter as before
visualize(() => new MovingAverage3(8))

在这个 Chisel 实例的可视化中，输入在左侧，z1 和 z2 寄存器用金色表示。寄存器和 io_in 都与它们的系数相乘，然后依次相加。`tail` 和 `bits` 元素用于防止加法过多增长。

你可能会问：“哦，做 Verilog 的事情在 Chisel 里也能做，那为什么我要使用 Chisel？”

我们很高兴你这么问！Chisel 的真正强大之处在于它能够创建**生成器，而不是实例**。假设我们不仅仅想创建一个 `MovingAverage3` 模块，而是想创建一个可以通过系数列表进行参数化的通用 `FIRFilter` 模块。

下面我们重写了 `MovingAverage3` 以接受一个系数序列。系数的数量将决定滤波器的大小。

In [ ]:
// Generalized FIR filter parameterized by the convolution coefficients
class FirFilter(bitWidth: Int, coeffs: Seq[UInt]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt())
  })
  // Create the serial-in, parallel-out shift register
  val zs = Reg(Vec(coeffs.length, UInt(bitWidth.W)))
  zs(0) := io.in
  for (i <- 1 until coeffs.length) {
    zs(i) := zs(i-1)
  }

  // Do the multiplies
  val products = VecInit.tabulate(coeffs.length)(i => zs(i) * coeffs(i))

  // Sum up the products
  io.out := products.reduce(_ +& _)
}

现在，通过在实例化时更改我们的 `coeffs` 参数，我们的 `FIRFilter` 模块可以用于实例化无数不同的硬件模块！下面我们创建三个不同的 `FIRFilter` 实例。

In [ ]:
// same 3-point moving average filter as before
visualize(() => new FirFilter(8, Seq(1.U, 1.U, 1.U)))

In [ ]:
// 1-cycle delay as a FIR filter
visualize(() => new FirFilter(8, Seq(0.U, 1.U)))

In [ ]:
// 5-point FIR filter with a triangle impulse response
visualize(() => new FirFilter(8, Seq(1.U, 2.U, 3.U, 2.U, 1.U)))

如果没有这种强大的参数化功能，我们将需要更多的模块定义，可能每个 FIR 滤波器都需要一个模块定义。理想情况下，我们希望我们的生成器是（1）可组合的，（2）强大的，以及（3）能够对生成的设计进行细粒度控制。

Chisel 的好处在于你如何使用它，而不是语言本身。
如果你决定编写实例而不是生成器，你会发现 Chisel 相对于 Verilog 的优势较少。
但是，如果你花时间学习如何编写生成器，那么 Chisel 的强大之处将变得明显，你会意识到你再也无法回到编写 Verilog 的时代。
学习编写生成器是困难的，但我们希望本教程能为你成为更好的硬件设计师、程序员和思考者铺平道路！

---
# All done!

[Return to the top.](#top)